<a href="https://colab.research.google.com/github/stiwari-ds/data-science-competitions/blob/main/zindi/trailblazers_open2all/notebooks/01_xgboost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [1]:
%%capture
!pip install --upgrade xgboost
!pip install --upgrade optuna

In [2]:
import os
import gc
import time
import warnings
import subprocess

gc.enable()
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('display.precision', 4)
np.set_printoptions(precision=4)

import xgboost
import optuna
optuna.logging.set_verbosity(optuna.logging.INFO)

from optuna.samplers import TPESampler
from optuna.pruners import HyperbandPruner
from optuna.integration import XGBoostPruningCallback
from xgboost import XGBRegressor

from sklearn.model_selection import KFold, GroupKFold
from sklearn.metrics import mean_squared_error

SEED = 23
os.environ['PYTHONHASHSEED'] = str(SEED)
np.random.seed(SEED)

In [3]:
#remove cell to run future versions
assert optuna.__version__ == '3.0.2', f'Change in Optuna version. Original notebook version: 3.0.2'
assert xgboost.__version__ == '1.6.2', f'Change in XGBoost version. Original notebook version: 1.6.2'

In [4]:
#Check GPU availability
try:
    subprocess.check_output('nvidia-smi')
    HAVE_GPU = True
except Exception:
    HAVE_GPU = False

print(f'GPU available: {HAVE_GPU}')

GPU available: True


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
DATA_PATH = '/content/drive/MyDrive/data_science_competitions/zindi/trailblazers_open2all/data'
train = pd.read_csv(f'{DATA_PATH}/raw/train.csv')
test = pd.read_csv(f'{DATA_PATH}/raw/test.csv')
sample_sub = pd.read_csv(f'{DATA_PATH}/raw/sample_sub.csv')

In [15]:
NOTEBOOK = '01'
SUBMISSION_PATH = f'/content/drive/MyDrive/data_science_competitions/zindi/trailblazers_open2all/submissions/nb_{NOTEBOOK}'
if not os.path.isdir(SUBMISSION_PATH):
    os.makedirs(SUBMISSION_PATH)

# Data Preparation

In [16]:
TARGET = train['target']
TEST_INDEX = test['Place_ID X Date'] #for submission files
GROUPS = np.array(train['Place_ID']) #for GroupKFold cross-validation

In [17]:
def preprocess(df: pd.DataFrame, is_train: bool = False) -> pd.DataFrame:
    #Convert date column to datetime type
    df['Date'] = pd.to_datetime(df['Date'])

    #Create date-based features
    df['month'] = df['Date'].dt.month
    df['day'] = df['Date'].dt.day
    df['day_of_week'] = df['Date'].dt.day_of_week

    #dropping non-feature columns
    df = df.drop(labels=['Place_ID X Date', 'Place_ID', 'Date'], axis=1)
    if is_train:
        df = df.drop(
            labels=['target', 'target_min', 'target_max', 'target_variance', 'target_count'], 
            axis=1
        )
    
    #reduce memory usage
    def reduce_mem(df: pd.DataFrame) -> pd.DataFrame:
        for col in df.columns:
            col_type = df[col].dtypes
            if col_type in ['int64', 'float64']:
                c_min = df[col].min()
                c_max = df[col].max()
                if str(col_type).startswith('int'):
                    if c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                        df[col] = df[col].astype(np.int16)
                    elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                        df[col] = df[col].astype(np.int32)
                    elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                        df[col] = df[col].astype(np.int64)
                else:
                    if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                        df[col] = df[col].astype(np.float16)
                    elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                        df[col] = df[col].astype(np.float32)
                    else:
                        df[col] = df[col].astype(np.float64)
        return df
    
    return reduce_mem(df)

In [18]:
train = preprocess(train, is_train=True)
test = preprocess(test)

gc.collect()

115

# Baseline

In [19]:
%%time
scores_rmse = []
cv = GroupKFold(n_splits=5)
X, y = train, TARGET
for fold, (train_idx, val_idx) in enumerate(cv.split(X, y, GROUPS)):
    X_train, y_train = X.loc[train_idx], y.iloc[train_idx]
    X_val, y_val = X.loc[val_idx], y.iloc[val_idx]

    model = XGBRegressor(
        objective='reg:squarederror',
        tree_method='gpu_hist' if HAVE_GPU else 'hist',
        eval_metric='rmse',
        early_stopping_rounds=50, 
        seed=SEED
    ) 
    model.fit(
        X_train, y_train,
        eval_set=[(X_val, y_val)],
        verbose=0
    )
    val_preds = model.predict(X_val)
    score = mean_squared_error(y_val, val_preds, squared=False)
    scores_rmse.append(score)
    print(f'Fold #{fold}: ({model.best_iteration} rounds) RMSE = {score:.5f}')
    _ = gc.collect()

print(f'\nAvg RMSE = {np.mean(scores_rmse):.5f} +/- {np.std(scores_rmse):.5f}\n')

Fold #0: (99 rounds) RMSE = 35.43706
Fold #1: (98 rounds) RMSE = 32.89088
Fold #2: (69 rounds) RMSE = 29.80391
Fold #3: (98 rounds) RMSE = 38.16264
Fold #4: (94 rounds) RMSE = 31.06374

Avg RMSE = 33.47165 +/- 3.01516

CPU times: user 5.43 s, sys: 325 ms, total: 5.76 s
Wall time: 5.26 s


# Hyperparameter tuning

In [32]:
def objective(trial, data, base_params):

    scores = []
    X, y = data

    param_grid = {
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, step=0.01),
        'max_depth': trial.suggest_int('max_depth', 3, 15),
        'min_child_weight': trial.suggest_int('min_child_weight', 2, 20),
        'gamma': trial.suggest_float('gamma', 0, 20, step=0.1), #complexity-control
        'alpha': trial.suggest_float('alpha', 0, 5, step=0.05), #L1-reg
        'lambda': trial.suggest_float('lambda', 1e-3, 1e5, log=True), #L2-reg
        # 'subsample': trial.suggest_float('subsample', 0.5, 1.0, step=0.05),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0, step=0.05),
        'colsample_bylevel': trial.suggest_float('colsample_bylevel', 0.5, 1.0, step=0.05),
        'colsample_bynode': trial.suggest_float('colsample_bynode', 0.5, 1.0, step=0.05)
    }

    cv = GroupKFold(n_splits=5)
    for fold, (train_idx, val_idx) in enumerate(cv.split(X, y, GROUPS)):
        X_train, y_train = X.loc[train_idx], y.iloc[train_idx]
        X_val, y_val = X.loc[val_idx], y.iloc[val_idx]
        model = XGBRegressor(
            **base_params, 
            **param_grid,
            callbacks=[XGBoostPruningCallback(
                trial=trial, 
                observation_key='validation_0-rmse')]
        )
        model.fit(
            X_train, y_train,
            eval_set=[(X_val, y_val)],
            verbose=0
        )
        preds = model.predict(X_val)
        scores.append(mean_squared_error(y_val, preds, squared=False))
    
    return np.mean(scores)

In [33]:
def tune_params(data, base_params, n_trials=10, direction='maximize'):
    study = optuna.create_study(
        sampler=TPESampler(seed=SEED),
        pruner=HyperbandPruner(),
        direction=direction
    )
    
    study.optimize(
        func=lambda trial: objective(trial, data, base_params),
        n_trials=n_trials,
        gc_after_trial=True
    )
    
    return study

# Cross-validation and experiment setup

In [34]:
def evaluate_model(data, model_params, verbose=True):
    oof_preds = {}  #out-of-fold predictions on train set
    test_preds = {} #predictions on test set for each fold
    scores_rmse = [] #RMSE scores on validation set

    X, X_test, y = data

    cv = GroupKFold(n_splits=5)
    for fold, (train_idx, val_idx) in enumerate(cv.split(X, y, GROUPS)):
        X_train, y_train = X.loc[train_idx], y.iloc[train_idx]
        X_val, y_val = X.loc[val_idx], y.iloc[val_idx]
        
        model = XGBRegressor(**model_params)
        model.fit(
            X_train, y_train,
            eval_set=[(X_val, y_val)],
            verbose=0
        )
        val_preds = model.predict(X_val)
        oof_preds.update(dict(zip(val_idx, val_preds)))
        test_preds[f'fold{fold}'] = model.predict(X_test)

        score = mean_squared_error(y_val, val_preds, squared=False)
        scores_rmse.append(score)
        if verbose:
            print(f'Fold #{fold}: ({model.best_iteration} rounds) RMSE = {score:.5f}')
        
        _ = gc.collect()

    print(f'\nAvg RMSE = {np.mean(scores_rmse):.5f} +/- {np.std(scores_rmse):.5f}')
    
    oof_preds = pd.Series(oof_preds).sort_index()
    print(f'OOF RMSE = {mean_squared_error(y, oof_preds, squared=False):.5f}')
    
    test_preds = pd.DataFrame.from_dict(test_preds)
    return oof_preds, test_preds

In [35]:
def run_experiment(data, n_trials=5):
        
    X, X_test, y = data
    
    base_params = {
        'objective': 'reg:squarederror',
        'n_estimators': 10000,
        'booster': 'gbtree',
        'eval_metric': 'rmse',
        'early_stopping_rounds': 100,
        'tree_method': 'gpu_hist' if HAVE_GPU else 'hist',
        'predictor': 'gpu_predictor' if HAVE_GPU else 'cpu_predictor',
        'verbosity': 1,
        'seed': SEED
    }
    
    print(f'---------------Hyperparameter tuning---------------')
    study = tune_params(
        data=(X, y), 
        base_params=base_params,
        n_trials=n_trials,
        direction='minimize'
    )
    print(f'Best trial: {study.best_trial.number} -> Best value(RMSE): {study.best_value:.5f}')
    print(f'Best hyperparameters:')
    for k, v in study.best_params.items():
        print(f'{k:20} - {v}')
    
    model_params = {**base_params, **study.best_params}
    print(f'-----------------Cross-validation------------------')
    oof_preds, test_preds = evaluate_model(
        data=(X, X_test, y), 
        model_params=model_params
    )
    return oof_preds, test_preds

### All features, No imputation

In [36]:
%%time
oof_preds, test_preds = run_experiment(data=(train, test, TARGET), n_trials=250)

[I 2022-09-24 05:30:40,624] A new study created in memory with name: no-name-9c65da42-d599-4d5d-920c-9387b277f7ce


---------------Hyperparameter tuning---------------


[I 2022-09-24 05:34:03,285] Trial 0 finished with value: 31.20323621889596 and parameters: {'learning_rate': 0.16, 'max_depth': 15, 'min_child_weight': 16, 'gamma': 5.6000000000000005, 'alpha': 1.1, 'lambda': 308.87067834937415, 'colsample_bytree': 0.55, 'colsample_bylevel': 0.7, 'colsample_bynode': 0.8}. Best is trial 0 with value: 31.20323621889596.
[I 2022-09-24 05:34:21,167] Trial 1 finished with value: 32.49035355912959 and parameters: {'learning_rate': 0.13, 'max_depth': 3, 'min_child_weight': 18, 'gamma': 17.7, 'alpha': 1.5, 'lambda': 52.078074880766074, 'colsample_bytree': 1.0, 'colsample_bylevel': 0.95, 'colsample_bynode': 0.5}. Best is trial 0 with value: 31.20323621889596.
[I 2022-09-24 05:34:54,794] Trial 2 finished with value: 31.839921833008184 and parameters: {'learning_rate': 0.09, 'max_depth': 6, 'min_child_weight': 17, 'gamma': 12.5, 'alpha': 0.55, 'lambda': 0.001009788660292811, 'colsample_bytree': 1.0, 'colsample_bylevel': 0.55, 'colsample_bynode': 0.7}. Best is tri

Best trial: 0 -> Best value(RMSE): 31.20324
Best hyperparameters:
learning_rate        - 0.16
max_depth            - 15
min_child_weight     - 16
gamma                - 5.6000000000000005
alpha                - 1.1
lambda               - 308.87067834937415
colsample_bytree     - 0.55
colsample_bylevel    - 0.7
colsample_bynode     - 0.8
-----------------Cross-validation------------------
Fold #0: (1026 rounds) RMSE = 33.31880
Fold #1: (821 rounds) RMSE = 30.42948
Fold #2: (621 rounds) RMSE = 27.55059
Fold #3: (663 rounds) RMSE = 36.33202
Fold #4: (521 rounds) RMSE = 28.38529

Avg RMSE = 31.20324 +/- 3.24535
OOF RMSE = 31.36913
CPU times: user 48min 5s, sys: 8.48 s, total: 48min 14s
Wall time: 47min 38s


In [38]:
for col in (test_preds.columns):
    sub = pd.DataFrame({'Place_ID X Date': TEST_INDEX, 'target': test_preds[col]})
    sub.to_csv(f'{SUBMISSION_PATH}/{col}.csv', index=False)

sub = pd.DataFrame({'Place_ID X Date': TEST_INDEX, 'target': test_preds.mean(axis=1)})
sub.to_csv(f'{SUBMISSION_PATH}/mean.csv', index=False)